In [1]:
from pandana.loaders import osm

bbox=[41.3752, 2.0354, 41.3889, 2.0586]

network = osm.network_from_bbox(bbox[0], bbox[1], bbox[2], bbox[3], network_type='drive')

In [2]:
import osm_query_pois

pois_dict = osm_query_pois.get_pois_dict(bbox)

In [26]:
for poi_key, poi_value in pois_dict.items():
    print(poi_key)
    print(poi_value)

shop
                          addr:city addr:country addr:housename  \
id                                                                
1429905107                      NaN          NaN            NaN   
2167329889                      NaN           ES            NaN   
2522597857  sant feliu de llobregat          NaN             18   
2702316473  Sant Feliu de Llobregat          NaN            NaN   
2702316474  Sant Feliu de Llobregat          NaN            NaN   
2702321499                      NaN          NaN            NaN   
2702321500                      NaN          NaN            NaN   
2702321501                      NaN          NaN            NaN   
2702321502                      NaN          NaN            NaN   
3757650540  Sant Feliu de Llobregat          NaN            NaN   
3757673776  Sant Feliu de Llobregat          NaN            NaN   
3757683127  Sant Feliu de Llobregat          NaN            NaN   
3757683129  Sant Feliu de Llobregat          NaN         

In [3]:
max_dist = 1000
max_pois=5

network.init_pois(num_categories=len(pois_dict), max_dist=max_dist, max_pois=max_pois)
nearest_dict = {}
for poi_key, poi_value in pois_dict.items():
    network.set_pois(poi_key, poi_value.lon, poi_value.lat)
    nearest_dict[poi_key] = network.nearest_pois(max_dist, poi_key, num_pois=max_pois)

In [10]:
for poi_key, nearest_poi_value in nearest_dict.items():
    print(poi_key)
    print(nearest_poi_value.describe())
    print('\n')

shop
                 1            2            3            4            5
count   430.000000   430.000000   430.000000   430.000000   430.000000
mean    307.109894   461.604492   558.622681   623.613586   702.627014
std     205.502350   214.547195   236.654419   221.498367   211.428589
min       0.000000     0.000000    39.035999    92.420998   199.960007
25%     173.450245   319.776245   390.881996   473.998497   558.084778
50%     275.285492   455.927002   535.804993   598.945496   680.078491
75%     397.348503   596.728256   740.121750   799.181503   882.919479
max    1000.000000  1000.000000  1000.000000  1000.000000  1000.000000


amenity
                 1            2            3            4            5
count   430.000000   430.000000   430.000000   430.000000   430.000000
mean    281.136383   414.676453   497.059052   550.569519   623.598999
std     209.355164   220.659256   221.811386   211.455017   210.042801
min       0.000000     0.000000     0.000000   107.539001   24

In [4]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from pandas import Series, DataFrame

from geo_node import GeoNode

nodes_df = network.nodes_df
    
digraph_df = DataFrame(list(
    [edge_row[1]['distance'],
     GeoNode(nodes_df.ix[edge_row[0][0]]),
     GeoNode(nodes_df.ix[edge_row[0][1]])]
    for edge_row in network.edges_df.iterrows()),
                       columns=['dist', 'from', 'to'])

digraph_df.groupby(['from', 'to']).mean() # avoid parallel edges

G = nx.from_pandas_dataframe(digraph_df, 'from', 'to', 'dist', create_using=nx.DiGraph())

In [6]:
# plot nodes in their positions
pos = {}
for node in G.nodes():
    pos[node] = node.pos    
    
# select only the largest component
G_sub = G.subgraph(max(nx.weakly_connected_components(G), key=len))

In [25]:
# plot
%matplotlib qt
# coloring G according to nearest pois
for poi_key, nearest_poi_value in nearest_dict.items():
    # check not only nearest_poi_value[1]
    plt.figure()
    plt.title(poi_key)
    nx.draw(G_sub, pos=pos, node_color=[nearest_poi_value[1][node.id] for node in G_sub])
# bmap, fig, ax = network.plot(nearest_bars[1], bbox=[bbox[0], bbox[1], bbox[2], bbox[3]])
# nx.draw(G, pos=pos, node_color=[node_coloring[node.id] for node in G])
# plt.show()

In [8]:
# histogram of node according to nearest bars
plt.hist(nearest_bars[1])
plt.show()